# Get summary GA

In [2]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tabulate import tabulate
from rdkit import Chem

from support_mvp.backup_plot_diagram.plot_diagram_schrock import (
    HARTREE2KCAL,
    base,
    collection,
    dicts,
    get_diagram_dicts,
    get_diagram_dicts_return_gibbs
)

oneup


In [3]:
smiles = ['CCCN1CCCC(N)(C2CCCC2)C1','CCCOC(=O)NCC1(N)CCCCC1', 'NC1(CCCCCl)CCCCC1', 'CC(C)(N)CCC1CCCCC1', 'CC(N)Cc1ccc(Cc2ccccc2)cc1', 'CC(N)Cc1ccc(CCl)cc1', 'CC(=O)Nc1ccc(F)c(C(=O)Cc2ccnc(N)c2)c1', 'C=C(C)CC(C)(C)N', 'CC(=O)c1ccc(F)c(C(=O)CN)c1', 'CC(=O)c1ccc(F)c(C(=O)N=CN)c1', 'CC(=O)c1cc(O)c(C(=O)CN)cc1O', 'N#Cc1ccnc(C#N)c1N', 'N#Cc1ccnc(C(=O)Cl)c1N', 'N#Cc1cnc(C#N)c(N)c1', 'N#Cc1ncc(CC(=O)O)cc1N', 'N#Cc1ncc(C(=O)CO)cc1N', 'Nc1ccccc1N=CC(=O)Cl', 'NCCc1ncnc2ccccc12', 'NCC=Cc1ncnc2ccccc12', 'NC=NC(=O)c1cccc(Br)c1', 'NCCCc1ncnc2ccccc12', 'NCCCOc1ccnc2cccnc12', 'NCCCOc1ncnc2cccnc12', 'NCCOC(=O)c1ncnc2ccccc12']

In [4]:
pkls = [
"/home/magstr/Documents/GB_GA/notebooks/df_nh3ton2.pkl",
"/home/magstr/Documents/GB_GA/notebooks/df_n2binding.pkl",
"/home/magstr/Documents/GB_GA/notebooks/df_reduction_step.pkl",
]

In [5]:
pd.read_pickle(pkls[0])

,smiles,final_dft_opt,scoring_function,postera_score,steps
6,CCNC(N)CCF,-35.40,NH3 -> N2,0.080245,1
8,CNC(N)Cc1ccccc1,-31.20,NH3 -> N2,0.000000,0
9,CNCCC1(N)CCCCC1,-31.15,NH3 -> N2,0.000000,0
11,CCC(N)CNc1cccc(S)c1,-29.09,NH3 -> N2,0.081222,1
13,CCCOC(=O)NCC1(N)CCCCC1,-28.64,NH3 -> N2,0.087442,1
...,...,...,...,...,...
114,NC1(C2CCCCC2)CCCCC1,-13.91,NH3 -> N2,0.000000,0
116,CNCC(N)c1ccc(CCl)cc1,-13.40,NH3 -> N2,0.090271,1
117,CC(N)Cc1ccc(CCl)cn1,-13.37,NH3 -> N2,0.081618,1
118,CCCCN1CCCC(N)(CC)C1,-12.82,NH3 -> N2,0.151494,1


In [6]:
base = Path("/home/magstr/Documents/")
path_dir = base / "schrock/diagrams_schrock/dft/candidate_cycle/"

In [7]:
path_dir

PosixPath('/home/magstr/Documents/schrock/diagrams_schrock/dft/candidate_cycle')

In [11]:
folders

[PosixPath('/home/magstr/Documents/schrock/diagrams_schrock/dft/candidate_cycle/dft_folder_41_44/NCCOC(=O)c1ncnc2ccccc12')]

In [10]:
d3 = {}
summary_dict = {}
for elem in smiles:
    
    
    m = Chem.MolFromSmiles(elem)
    # Replace tricky primary amines in the frag:
    prim_match = Chem.MolFromSmarts("[NX3;H2]")
    rm = Chem.ReplaceSubstructs(
                                m,
                                prim_match,
                                Chem.MolFromSmiles("[1*]"),
                                replaceAll=True,
                            )[0]
    new_smiles=Chem.MolToSmiles(rm
                               )
    
    for f in pkls:
        mol_df = pd.read_pickle(f)
    # Get the scoring
        tmp = mol_df.loc[mol_df["smiles"] == elem]
        if not tmp.empty:
            summary_dict[new_smiles] = {'scoring function':tmp["scoring_function"].values[0]}

            break
        else:
            val = ""
            continue

    folders = sorted(path_dir.rglob(f"dft*/{elem}"))
    
    options = {
                "exchange": True,
                "thermal_correction": False,
                'diagrams': [
                    [
                        folders[0],
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca"],
                    ],
                ]
            }
    options = defaultdict(lambda: False, options)
    compare_data, delta_gibbs = get_diagram_dicts_return_gibbs(options)
    
    options_ref = {
                "exchange": True,
                "thermal_correction": False,
                'diagrams': [
                    [
                        base / "schrock/diagrams_schrock/dft/cycle_orca_fullyconverged_sarcJ",
                        "orca_stepsize.out",
                        "FINAL SINGLE",
                        dicts["parts_orca"],
                    ],
                ]
            }
    

    options_ref = defaultdict(lambda: False, options_ref)
    compare_data_ref, delta_gibbs_ref = get_diagram_dicts_return_gibbs(options_ref)
    
    
    summary_dict[new_smiles]['NH3toN2'] = compare_data[elem]['Mo_N2_exchange']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3toN2_NH3'] = compare_data[elem]['Mo_N2_NH3']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3+toNH3'] = compare_data[elem]['Mo_NH3']-compare_data[elem]['Mo_NH3+']
    
    
    delta_gibbs_ref.pop('Mo_N2_exchange')
    delta_gibbs_ref.pop('Mo_N2_bare')
    delta_gibbs_ref.pop('Mo')
    
    delta_gibbs.pop('Mo_N2_exchange')
    delta_gibbs.pop('Mo_N2_bare')
    delta_gibbs.pop('Mo')
    
    # Find step that increased most compared to refference
    d3 = {key: delta_gibbs[key] - delta_gibbs_ref.get(key, 0) for key in delta_gibbs}
    summary_dict[new_smiles]['max_change'] = d3[max(d3, key=d3.get)]

Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/candidate_cycle/dft_folder_16_20/CCCN1CCCC(N)(C2CCCC2)C1
NH3->N2 : -24.313430663663894
NH3->N2_NH3 : 39.4355510068126
NH3+->NH3 : 21.39036721480079
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/cycle_orca_fullyconverged_sarcJ
NH3->N2 : -11.18007980659604
NH3->N2_NH3 : -13.813871419988573
NH3+->NH3 : 10.469372462714091
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/candidate_cycle/dft_folder_16_20/CCCOC(=O)NCC1(N)CCCCC1
NH3->N2 : -12.4087695479393
NH3->N2_NH3 : 0.37337522953748703
NH3+->NH3 : 16.81849132082425
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/cycle_orca_fullyconverged_sarcJ
NH3->N2 : -11.18007980659604
NH3->N2_NH3 : -13.813871419988573
NH3+->NH3 : 10.469372462714091
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/candidate_cycle/dft_folder_26_30/NC1(CCCCCl)CCCCC1
NH3->N2 : -21.780213001184165
NH3->N2_NH3 : 63.696127072907984
NH3+->NH

In [39]:
for k, v in summary_dict.items():
        print(k,v)

[1*]C1(C2CCCC2)CCCN(CCC)C1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -24.313000000000002, 'NH3toN2_NH3': 39.436, 'NH3+toNH3': 21.39, 'max_change': 53.25}
[1*]C1(CNC(=O)OCCC)CCCCC1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -12.408999999999992, 'NH3toN2_NH3': 0.37300000000000466, 'NH3+toNH3': 16.817999999999998, 'max_change': 14.187}
[1*]C1(CCCCCl)CCCCC1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -21.78, 'NH3toN2_NH3': 63.696, 'NH3+toNH3': 17.381, 'max_change': 77.50999999999999}
[1*]C(C)(C)CCC1CCCCC1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -21.608999999999995, 'NH3toN2_NH3': -8.471999999999994, 'NH3+toNH3': 19.578000000000003, 'max_change': 35.533}
[1*]C(C)Cc1ccc(Cc2ccccc2)cc1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -24.031999999999996, 'NH3toN2_NH3': -6.185999999999993, 'NH3+toNH3': 19.057000000000002, 'max_change': 8.911000000000001}
[1*]C(C)Cc1ccc(CCl)cc1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -23.644000000000005, 'NH3toN2_NH3': -10.010000000000005, 'NH3+toNH3

In [42]:
def create_latex_tables():

    # Example code:
    # https://colab.research.google.com/drive/1Iq10lHznMngg1-Uoo-QtpTPii1JDYSQA?usp=sharing#scrollTo=K7NNR1Vg40Vo

    # Simple header
    headers = ["Smiles","Scoring", "NH3toN2", 'NH3+toNH3', 'NH3toN2_NH3', 'Max \delta\delta E']
    
    rows = [ ]
    for k, v in summary_dict.items():
        rows.append([k,v['NH3toN2'],v['NH3+toNH3'],v['NH3toN2_NH3'], v['max_change']])


    print("Tabulate Table:")
    print(tabulate(rows, headers=headers))

    # Formatting option: floatfmt=".3f"
    print("\nTabulate Latex:")
    print(tabulate(rows, headers=headers, tablefmt="latex", floatfmt=".2f"))

    return

In [43]:
create_latex_tables()

Tabulate Table:
Smiles                                    Scoring    NH3toN2    NH3+toNH3    NH3toN2_NH3
--------------------------------------  ---------  ---------  -----------  -------------
[1*]C1(C2CCCC2)CCCN(CCC)C1                -24.313     21.39        39.436         53.25
[1*]C1(CNC(=O)OCCC)CCCCC1                 -12.409     16.818        0.373         14.187
[1*]C1(CCCCCl)CCCCC1                      -21.78      17.381       63.696         77.51
[1*]C(C)(C)CCC1CCCCC1                     -21.609     19.578       -8.472         35.533
[1*]C(C)Cc1ccc(Cc2ccccc2)cc1              -24.032     19.057       -6.186          8.911
[1*]C(C)Cc1ccc(CCl)cc1                    -23.644     16.732      -10.01           6.263
[1*]c1cc(CC(=O)c2cc(NC(C)=O)ccc2F)ccn1     -3.492      8.552      -13.451         40.735
[1*]C(C)(C)CC(=C)C                        -18.511     17.119       -7.977          8.892
[1*]CC(=O)c1cc(C(C)=O)ccc1F                10.518     -3.033       -3.824         21.096
[1*]C=N